In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

In [20]:
# Just using 2009 data for this specific training model -- we can generalize to all matches eventually 
match2009 = pd.read_csv("match2009.csv")
match2009 = match2009.drop(['Match Date', 'Score', 'Player 1 Ranking', 'Player 2 Ranking', 'Player 1 Name', 'Player 2 Name'], axis=1)
match2009.head(5)

,Winning player name,Player 1 Aces,Player 2 Aces,Player 1 Double Faults,Player 2 Double Faults,Player 1 1st Serve Percentage,Player 2 1st Serve Percentage,Player 1 1st Serve Points Won,Player 2 1st Serve Points Won,Player 1 2nd Serve Points Won,...,Player 1 Max Games in Row,Player 2 Max Games in Row,Player 1 Service Games Won,Player 2 Service Games Won,Player 1 Return Games Won,Player 2 Return Games Won,Player 1 Total Games Won,Player 2 Total Games Won,Player 1 Distance Covered,Player 2 Distance Covered
0,R. Federer,4.9%,1.1%,2.5%,3.4%,69.1%,74.2%,78.6%,65.2%,60.0%,...,NaN,NaN,93.3%,78.6%,21.4%,6.7%,58.6%,41.4%,NaN,NaN
1,N. Davydenko,4.4%,10.4%,1.1%,3.9%,54.4%,64.9%,69.4%,82.0%,65.9%,...,NaN,NaN,86.7%,73.3%,26.7%,13.3%,56.7%,43.3%,NaN,NaN
2,N. Davydenko,12.3%,14.0%,3.5%,0.0%,66.7%,63.2%,86.8%,72.2%,57.9%,...,NaN,NaN,100.0%,77.8%,22.2%,0.0%,63.2%,36.8%,NaN,NaN
3,R. Federer,9.0%,8.9%,1.5%,7.9%,64.2%,45.5%,86.0%,71.7%,37.5%,...,NaN,NaN,83.3%,61.5%,38.5%,16.7%,60.0%,40.0%,NaN,NaN
4,J. M. Del Potro,13.3%,17.5%,1.0%,3.9%,71.4%,69.9%,77.1%,79.2%,64.3%,...,NaN,NaN,94.1%,87.5%,12.5%,5.9%,54.5%,45.5%,NaN,NaN


In [21]:
## Prepping the data to be usable for logistic regression between certain features and whether the player won or lost 

match1 = match2009[match2009.columns.drop(match2009.filter(regex='Player 2').columns)]
match2 = match2009[match2009.columns.drop(match2009.filter(regex='Player 1').columns)]

match1['winning'] = 1
match2['winning'] = 0

# Associated with the columns
values = ["Aces", "Double Faults", "1st Serve Percentage", "Return Points Won"]
match1 = match1[['winning', 'Player 1 Aces', 'Player 1 Double Faults', 'Player 1 1st Serve Percentage', 'Player 1 Return Points Won']]
match2 = match2[['winning', 'Player 2 Aces', 'Player 2 Double Faults', 'Player 2 1st Serve Percentage', 'Player 2 Return Points Won']]

match1 = match1.rename(columns={"Player 1 Aces": "Aces", "Player 1 Double Faults": "DoubleFaults", "Player 1 1st Serve Percentage": "firstServePercentage", "Player 1 Return Points Won": "ReturnPointsWon"})
match2 = match2.rename(columns={"Player 2 Aces": "Aces", "Player 2 Double Faults": "DoubleFaults", "Player 2 1st Serve Percentage": "firstServePercentage", "Player 2 Return Points Won": "ReturnPointsWon"})

# merges the values such that we have a dataframe consisting of all the values, associated with either a win or a loss 
matches = match1.merge(match2, how='outer') 
matches

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,winning,Aces,DoubleFaults,firstServePercentage,ReturnPointsWon
0,1,4.9%,2.5%,69.1%,38.2%
1,1,4.4%,1.1%,54.4%,32.5%
2,1,12.3%,3.5%,66.7%,38.6%
3,1,9.0%,1.5%,64.2%,46.5%
4,1,13.3%,1.0%,71.4%,32.0%
...,...,...,...,...,...
5427,0,10.5%,5.3%,59.2%,30.9%
5428,0,7.5%,4.5%,64.2%,25.4%
5429,0,7.3%,3.6%,65.5%,27.1%
5430,0,5.7%,10.0%,58.6%,28.3%


In [22]:
matches['Aces'] = pd.to_numeric(matches['Aces'].str.replace('%', ''))
matches['DoubleFaults'] = pd.to_numeric(matches['DoubleFaults'].str.replace('%', ''))
matches['firstServePercentage'] = pd.to_numeric(matches['firstServePercentage'].str.replace('%', ''))
matches['ReturnPointsWon'] = pd.to_numeric(matches['ReturnPointsWon'].str.replace('%', ''))
matches = matches.dropna()

In [23]:
## Logistic Regression 

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# setting a diagnosis vector 
y = matches['winning']

# Creating a design matrix to fit our data (use specific column values)
X = matches.iloc[:,[1,2,3,4]].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, random_state=0)


# Fitting a logistic regression model predicting y using X 
m = LogisticRegression()
f = m.fit(X_train,y_train)

# Predictions: 
pred = f.predict(X)

# Computes the confusion matrix 
cm = confusion_matrix(y, pred)
print("Confusion Matrix: " + str(cm))

# Accuracy 
accuracy = accuracy_score(y, pred)
print("Accuracy: " + str(accuracy))

# F-Score 
fScore = f1_score(y, pred)
print("F-Score: " + str(fScore))

# One way to view the summary: 
logit_model = sm.Logit(y,X)
result = logit_model.fit()
print(result.summary2())

# Other way to view the summary: 
m1 = smf.logit("winning ~ Aces", data=matches).fit()
m2 = smf.logit("winning ~ DoubleFaults + Aces + firstServePercentage + ReturnPointsWon", data=matches).fit()
m1.summary()
m2.summary()

Confusion Matrix: [[713 153]
 [156 710]]
Accuracy: 0.8215935334872979
F-Score: 0.8212839791787161
Optimization terminated successfully.
         Current function value: 0.474116
         Iterations 6
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.316      
Dependent Variable: winning          AIC:              1650.3389  
Date:               2020-12-06 21:06 BIC:              1672.1670  
No. Observations:   1732             Log-Likelihood:   -821.17    
Df Model:           3                LL-Null:          -1200.5    
Df Residuals:       1728             LLR p-value:      3.8722e-164
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
---------------------------------------------------------------------
       Coef.     Std.Err.       z        P>|z|      [0.025     0.975]
---------------------------------------------------------------------
x1     0.0891

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                winning   No. Observations:                 1732
Model:                          Logit   Df Residuals:                     1727
Method:                           MLE   Df Model:                            4
Date:                Sun, 06 Dec 2020   Pseudo R-squ.:                  0.4575
Time:                        21:06:35   Log-Likelihood:                -651.28
converged:                       True   LL-Null:                       -1200.5
Covariance Type:            nonrobust   LLR p-value:                1.598e-236
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              -13.1003      0.861    -15.222      0.000     -14.787     -11.413
DoubleFaults            -0.1247      0.031     -4.065      0.000      -0.185      -0.065
Aces                     0.1522      0.014     11.084      0.000       0.125       0.179
firstServePercentage     0.0296      0.009      3.270      0.001       0.012       0.047
ReturnPointsWon          0.2931      0.015     19.740      0.000       0.264       0.322
========================================================================================
"""